In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
dataset = pd.read_csv('dataset.csv')
dataset.set_index(['date','sex','age','height','timezone'], inplace=True)
dataset.head()

In [ ]:
test_split=round(len(dataset)*0.20)
df_for_training=dataset[:-test_split]
df_for_testing=dataset[-test_split:]
print(df_for_training.shape)
print(df_for_testing.shape)

In [ ]:
#Normalizzazione_internet
scaler = MinMaxScaler(feature_range=(0,1))
df_for_training_scaled = scaler.fit_transform(df_for_training)
df_for_testing_scaled= scaler.transform(df_for_testing)
df_for_training_scaled

In [ ]:
#n_past is the number of step we will look in the past to predict the next target value.
col2pred = 3  # 0:steps, 1:bedin, 2:bedout, 3:sleep duration, 4:deep duration, 5:lightduration
def createXY(dataset,n_past):
    dataX = []
    dataY = []
    for i in range(n_past, len(dataset)):
            #per ogni feature fa l'append dei precedenti
            dataX.append(dataset[i - n_past:i, 0:dataset.shape[1]])
            
            dataY.append(dataset[i,col2pred])
    return np.array(dataX),np.array(dataY)

trainX,trainY=createXY(df_for_training_scaled,30)
testX,testY=createXY(df_for_testing_scaled,30)

In [ ]:
# features num
dataset.shape[1]

In [ ]:
#Creazione del modello
model = Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(30,6)))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss = 'mse', optimizer = 'adam')
model.summary()

In [ ]:
history = model.fit(trainX, trainY, epochs = 8, verbose = 1, batch_size = 4096) # 8192

In [ ]:
prediction=model.predict(testX)
print("prediction\n", prediction)
print("\nPrediction Shape-",prediction.shape)

In [ ]:
prediction_copies_array = np.repeat(prediction,6, axis=-1)
pred=scaler.inverse_transform(np.reshape(prediction_copies_array,(len(prediction),6)))[:,col2pred]

In [ ]:
original_copies_array = np.repeat(testY,6, axis=-1)
original=scaler.inverse_transform(np.reshape(original_copies_array,(len(testY),6)))[:,col2pred]

In [ ]:
print("Pred Values-- " ,pred)
print("\nOriginal Values-- " ,original)

In [ ]:
plt.plot(original[:1250], color = 'red', label = 'Real sleep duration')
plt.plot(pred[:1250], color = 'blue', label = 'Predicted sleep duration')
plt.title('sleep duration Prediction')
plt.xlabel('Time')
plt.ylabel('steps')
plt.legend()
plt.show()

In [ ]:
#Normalizzazione
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

x_data = x_scaler.fit_transform(dataset[['steps','bedin','bedout','sleep duration','deepduration','lightduration']])
y_data = y_scaler.fit_transform(dataset[['sleep duration']])